In [1]:
import torch
import torch.nn as nn
from torchvision import datasets,utils
import torch.utils.data as data
import torchvision.transforms as transforms
from torchvision import models
import torch.nn.functional as F
from torch.nn import init
import torchvision as tv
from torch.optim import lr_scheduler
from pytorch_msssim import SSIM


import datetime
import time
from tqdm import tqdm 
import pandas as pd

import shutil 
import ntpath
import numpy as np
from PIL import Image
import os
import datetime
import time
from pathlib import Path
from sklearn.metrics import roc_auc_score, roc_curve

# from sklearn.metrics import precision_score,f1_score,roc_curve,auc,accuracy_score
# from models.gaussian_blur import GaussianBlur
# from models.gamma_correction import GammaCorrection,HistogramEqualization


import matplotlib
import matplotlib.pyplot as plt 
device = torch.device("cuda:6" if torch.cuda.is_available() else "cpu")

In [74]:
class Config(object):
    def __init__(self):
        self.name = 'FM_MIMIC/EXP4'
        self.dataset_name = 'MIMIC'
        self.dataroot ='../DATASET/MIMIC/IMAGES'
        self.save_path = './checkpoint/' + self.name
        self.model_path = self.save_path + '/models'
        self.decode_path = self.save_path + '/decoded_results'
        self.val_path = self.save_path + '/val_results'
        self.test_path = self.save_path + '/test_results'
        
        self.num_threads = 1
        self.shuffle_dataset=True
        self.random_seed=24


        self.lr = 0.0001     
        
        self.serial_batches = False
        self.phase='train'
        
        self.batch_size = 1
        self.test_batch_size = 1
        self.c_batch_size = 1
        self.max_epochs = 500
        self.save_every = 1     #epoch
        self.plot_every = 1    # epoch to save decoded images

        os.makedirs(self.save_path, exist_ok=True)
        os.makedirs(self.model_path, exist_ok=True)
        os.makedirs(self.decode_path, exist_ok=True)
        os.makedirs(self.val_path, exist_ok=True)
        os.makedirs(self.test_path, exist_ok=True)
opt = Config()

In [3]:
# path= '../DATASET/MIMIC/test.csv'
# df = pd.read_csv(path)
# df = df.sample(frac = 1) 
# df.to_csv('../DATASET/MIMIC/test.csv',index=False)

In [4]:
phases = ['train', 'val','test','covid'] 
root_dir = '../DATASET/MIMIC/'
data_dir ='../DATASET/MIMIC/IMAGES'
# Path to csvfiles on training,validation and tetsing data
csvpath = {phase: f'{root_dir}/{phase}.csv' for phase in phases}
# Load data into dictionary of three Pandas DataFrames
dframe = {phase: pd.read_csv(csvpath[phase]) for phase in phases}
# Calculate sizes
dataset_sizes = {phase: len(dframe[phase]) for phase in phases}
print(dframe['train'].shape, dframe['val'].shape,dframe['test'].shape,dframe['covid'].shape)

(17909, 5) (5371, 5) (3586, 5) (392, 5)


In [5]:
num_classes = 4
class_names = dframe['train'].iloc[:, 1:].columns
print(class_names)
# indices we will calculate AUC for, 
competition_tasks = torch.ByteTensor([1,1,1,1]).bool()

df = dframe['train'].iloc[:, 1:].copy()
labels_array = df.iloc[:, 1:].copy()
labels_array = {phase: dframe[phase].iloc[:, 1:].copy() for phase in phases}

for phase in labels_array.keys():
    labels_array[phase] = torch.FloatTensor(labels_array[phase].to_numpy())
#     labels_array[phase] = torch.LongTensor(labels_array[phase].to_numpy())


print(labels_array['train'].shape)
print(labels_array['val'].shape)
print(labels_array['test'].shape)
print(labels_array['covid'].shape)

Index(['c', 'o', 'e', 'n'], dtype='object')
torch.Size([17909, 4])
torch.Size([5371, 4])
torch.Size([3586, 4])
torch.Size([392, 4])


In [6]:
labels_array['train'][0]

tensor([0., 1., 1., 0.])

In [7]:
labels_array['covid'][0]

tensor([0., 1., 0., 0.])

In [8]:
CLASS_NAMES = [ 'Consolidation', 'Opacity','Effusion', 'No_Findings']

In [75]:
transform=transforms.Compose([#transforms.Grayscale(num_output_channels=1),
                              transforms.Resize(256),
                              transforms.CenterCrop(224),
                              transforms.RandomHorizontalFlip(),
                              transforms.ToTensor(),
#                               transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

                            ])

# normalize = transforms.Normalize([0.485, 0.456, 0.406],
#                                      [0.229, 0.224, 0.225])
# transform=transforms.Compose([#transforms.Grayscale(num_output_channels=1),
#                               transforms.Resize(256),
#                               transforms.FiveCrop(224),
#                               transforms.Lambda
#                                 (lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
#                               transforms.Lambda
#                                 (lambda crops: torch.stack([normalize(crop) for crop in crops]))
#                             ])
           

class Data(data.Dataset):
    def __init__(self,img_paths, labels_array ,phase=None, tforms=None):
        self.transform=transform
        self.phase = phase
        self.img_paths = img_paths
        self.labels_array = labels_array 
                    
    def __len__(self):
            return len(self.img_paths)
 
    def __getitem__(self, index):
        file = self.img_paths[index]
        image = Image.open(file).convert('RGB')
        if self.transform is not None:
            image = self.transform(image)

        label = self.labels_array[index]
#         label = torch.FloatTensor([label])
#         label = torch.LongTensor([label])
        return image,label

In [76]:
img_paths = {phase: data_dir +'/'+ dframe[phase].iloc[:, 0] for phase in phases}
datasets = {phase: Data(img_paths=img_paths[phase],labels_array=labels_array[phase],phase=phase,tforms=transform) 
            for phase in phases}

In [77]:
num_workers = 8
params = {'train': {'batch_size': opt.batch_size,
                    'shuffle': True,
                    'num_workers': num_workers,
                    'pin_memory': True,
                    'sampler': None,
                    'drop_last':True},
          'val': {'batch_size': opt.batch_size,
                  'shuffle': True,
                  'num_workers': num_workers,
                  'pin_memory': True,
                   'drop_last':True},
           'test': {'batch_size': opt.test_batch_size,
                  'shuffle': True,
                  'num_workers': num_workers,
                  'pin_memory': True,
                   'drop_last':True},
            'covid': {'batch_size': opt.c_batch_size,
                  'shuffle': True,
                  'num_workers': num_workers,
                  'pin_memory': True,
                   'drop_last':True}
         }

if params['train']['sampler'] is not None:
    params['train']['shuffle'] = False
    
dataloaders = {phase: data.DataLoader(datasets[phase], **params[phase]) for phase in phases}
for phase in phases:
    print(f'{len(datasets[phase])} images loaded under {phase}')

17909 images loaded under train
5371 images loaded under val
3586 images loaded under test
392 images loaded under covid


In [12]:
# dataiter = iter(dataloaders['test'])
# images, labels = dataiter.next()
# # grid_img = torchvision.utils.make_grid(images, nrow=1)
# plt.imshow(images.permute(1, 2, 0))
# print(labels)

In [13]:
# model=models.densenet169(pretrained=True)
# num_ftrs = model.classifier.in_features
# model.classifier= nn.Linear(num_ftrs, num_classes)

In [14]:
class DenseNet121(nn.Module):
    """Model modified.
    The architecture of our model is the same as standard DenseNet121
    except the classifier layer which has an additional sigmoid function.
    """
    def __init__(self, out_size):
        super(DenseNet121, self).__init__()
        self.densenet121 = models.densenet121(pretrained=True)
        num_ftrs = self.densenet121.classifier.in_features
        self.densenet121.classifier = nn.Sequential(
            nn.Linear(num_ftrs, out_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.densenet121(x)
        return x

In [15]:
# cnn_model = torch.nn.Sequential(*(list(model.children())[:-3]))

In [16]:
# dataiter = iter(dataloaders['train'])
# images, labels = dataiter.next()
# out = cls(images)
# print(out.shape)

In [17]:
model = DenseNet121(4)
state = torch.load(os.path.join(opt.model_path,'chexnet2.pth'),map_location='cuda:6')
model.load_state_dict(state)

<All keys matched successfully>

In [18]:
def plot_roc_curve(fpr, tpr):
    plt.plot(fpr, tpr, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()


def compute_AUCs(gt, pred):
    """Computes Area Under the Curve (AUC) from prediction scores.
    Args:
        gt: Pytorch tensor on GPU, shape = [n_samples, n_classes]
          true binary labels.
        pred: Pytorch tensor on GPU, shape = [n_samples, n_classes]
          can either be probability estimates of the positive class,
          confidence values, or binary decisions.
    Returns:
        List of AUROCs of all classes.
    """
    AUROCs = []
    TPRs = FPRs = THs = []
    gt_np = gt.cpu().numpy()
    pred_np = pred.cpu().numpy()
    for i in range(num_classes):
        try:
            AUROCs.append(roc_auc_score(gt_np[:, i], pred_np[:, i]))
            fpr, tpr, thresholds = roc_curve(gt_np[:, i], pred_np[:, i])
            TPRs.append(tpr)
            FPRs.append(fpr)
            THs.append(thresholds)
        except ValueError:
            pass
    return AUROCs

In [19]:
def down_pooling():
    return nn.MaxPool2d(2)

class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.n_z=512      # number of dimensions in latent space.
        
        self.conv1 = nn.Sequential(nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3,stride=1,padding=1),
                                 nn.BatchNorm2d(64),
                                 nn.ReLU(inplace=True)
                                )
        self.conv11 = nn.Sequential(nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3,stride=1,padding=1),
                                 nn.BatchNorm2d(64),
                                 nn.ReLU(inplace=True)
                                )
        
        self.conv2=nn.Sequential(nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3,stride=1,padding=1),
                                 nn.BatchNorm2d(128),
                                 nn.ReLU(inplace=True)
                                )
        self.conv22=nn.Sequential(nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3,stride=1,padding=1),
                                 nn.BatchNorm2d(128),
                                 nn.ReLU(inplace=True)
                                )
        self.conv3=nn.Sequential(nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3,stride=1,padding=1),
                                 nn.BatchNorm2d(256),
                                 nn.LeakyReLU(inplace=True)
                                )
        self.conv33=nn.Sequential(nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3,stride=1,padding=1),
                                 nn.BatchNorm2d(256),
                                 nn.ReLU(inplace=True)
                                )
        self.conv4=nn.Sequential(nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3,stride=1,padding=1),
                                 nn.BatchNorm2d(512),
                                 nn.ReLU(inplace=True)
                                )
        self.conv44=nn.Sequential(nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3,stride=1,padding=1),
                                 nn.BatchNorm2d(512),
                                 nn.ReLU(inplace=True)
                                )        
        self.conv5=nn.Sequential(nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3,stride=1,padding=1),
                                 nn.BatchNorm2d(1024),
                                 nn.ReLU(inplace=True)
                                )
        self.conv55=nn.Sequential(nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3,stride=1,padding=1),
                                 nn.BatchNorm2d(1024),
                                 nn.ReLU(inplace=True)
                                )
        self.down_pooling = nn.MaxPool2d(2)
        
#         self.mu = nn.Linear(1024*14*14, 512)
#         self.logvar = nn.Linear(1024*14*14, 512)
        self.mu = nn.Sequential( nn.Linear((1024*14*14), 512),
                                 nn.BatchNorm1d(512),
                                 nn.ReLU(inplace=True),
                                 nn.Linear(512,512))
        self.logvar = nn.Sequential( nn.Linear((1024*14*14),512),
                                 nn.BatchNorm1d(512),
                                 nn.ReLU(inplace=True),
                                 nn.Linear(512,512))



        
    
    def reparameterize(self, mu, log_var):
        """
        :param mu: mean from the encoder's latent space
        :param log_var: log variance from the encoder's latent space
        """
        std = torch.exp(0.5*log_var) # standard deviation
        eps = torch.randn_like(std) # `randn_like` as we need the same size
        sample = mu + (eps * std) # sampling as if coming from the input space
        return sample

        
    def forward(self,x):
        
        x=self.conv1(x)
        x=self.conv11(x)
        x = self.down_pooling(x)
        
        x=self.conv2(x)
        x=self.conv22(x)
        x = self.down_pooling(x)
        
        
        x=self.conv3(x)
        x=self.conv33(x)
        x = self.down_pooling(x)

        
        x=self.conv4(x)
        x=self.conv44(x)
        x = self.down_pooling(x)
        
        x=self.conv5(x)
        x=self.conv55(x)
#         print(x.size())
        
        x = x.view(x.size(0),-1)
        mu, logvar = self.mu(x), self.logvar(x)
        x = self.reparameterize(mu, logvar)
        
        return x,mu,logvar



# In[ ]:


def up_pooling(in_channels, out_channels, kernel_size=2, stride=2):
    return nn.Sequential(
        nn.ConvTranspose2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True)
    )
def conv_bn_leru(in_channels, out_channels, kernel_size=3, stride=1, padding=1):
    return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
    )

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        
#         self.fc = nn.Sequential(nn.Linear(512,1024*14*14),
#                                 nn.ReLU()
#                                 )
        self.fc = nn.Sequential( nn.Linear(512, 512),
                                 nn.BatchNorm1d(512),
                                 nn.ReLU(inplace=True),
                                 nn.Linear(512,1024*14*14))
    
        self.up_pool6 = up_pooling(1024, 1024)
        self.conv6 = conv_bn_leru(1024, 512)
        self.up_pool7 = up_pooling(512, 512)
        self.conv7 = conv_bn_leru(512, 256)
        self.up_pool8 = up_pooling(256, 256)
        self.conv8 = conv_bn_leru(256, 128)
        self.up_pool9 = up_pooling(128, 128)
        self.conv9 = conv_bn_leru(128, 64)

        self.conv10 = nn.Conv2d(64, 3,1)
        self.relu = nn.ReLU()
    
        
    def forward(self,x):
        x  = self.fc(x)
        x5 = x.view(x.size(0),1024,14,14)
        
        p6 = self.up_pool6(x5)
        x6 = self.conv6(p6)

        p7 = self.up_pool7(x6)
        x7 = self.conv7(p7)

        p8 = self.up_pool8(x7)
        x8 = self.conv8(p8)

        p9 = self.up_pool9(x8)
        x9 = self.conv9(p9)
        
        output = self.conv10(x9)
        output = self.relu(output)
        return output



In [20]:
def weight_init(m):
    classname = m.__class__.__name__
    if isinstance(m, nn.Conv2d):
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)
    if isinstance(m, nn.Linear):
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)
    if isinstance(m, nn.BatchNorm2d):
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)


encoder=Encoder()
encoder.apply(weight_init)
decoder=Decoder()
decoder.apply(weight_init)
encoder, decoder= encoder.to(device), decoder.to(device)

In [21]:
class SSIM_Loss(SSIM):
    def forward(self, img1, img2):
        return ( 1 - super(SSIM_Loss, self).forward(img1, img2)) 


In [22]:
def free_params(module: nn.Module):
    for p in module.parameters():
        p.requires_grad = True

def frozen_params(module: nn.Module):
    for p in module.parameters():
        p.requires_grad = False

In [23]:
frozen_params(encoder)
frozen_params(decoder)

In [24]:
state = torch.load(os.path.join(opt.model_path,'model2.pth'),map_location='cuda:6')
model.load_state_dict(state['state_dict'])
print("Loaded pre-trained model with success.")
e_counter=state['epoch']
best_auc = state['valid_acc_max']
print('Previously Trained for {} epoches'.format(e_counter))
print('Best val AUC till yet :',best_auc)

Loaded pre-trained model with success.
Previously Trained for 197 epoches
Best val AUC till yet : 0.8380716209476995


In [25]:
state = torch.load(os.path.join(opt.model_path, 'vae.pth'),map_location='cuda:6')
encoder.load_state_dict(state['enc_state_dict'])
decoder.load_state_dict(state['dec_state_dict'])
print("Loaded pre-trained models with success.")
e_counter=state['epoch']
best_valid_loss = state['loss_min']
print('Previously Trained for {} epoches'.format(e_counter))

Loaded pre-trained models with success.
Previously Trained for 866 epoches


In [26]:
def train_MainNet(**kwargs):
    torch.cuda.empty_cache()
    opt = Config()
    print('loading the model...') 
#     criterion = nn.BCEWithLogitsLoss(reduction='sum').to(device)
    criterion = nn.BCELoss().to(device)
    l1_loss = nn.L1Loss()

    
    model.to(device)
    l=10
    optimizer = torch.optim.Adam(model.parameters(), lr = opt.lr,weight_decay=4e-3)
    scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=opt.max_epochs, eta_min=0.000001)

    try:
        state = torch.load(os.path.join(opt.model_path,'I_model.pth'),map_location='cuda:6')
        model.load_state_dict(state['state_dict'])
        print("Loaded pre-trained model with success.")
        e_counter=state['epoch']
        best_auc = state['valid_acc_max']
        print('Previously Trained for {} epoches'.format(e_counter))
        print('Best val AUC till yet :',best_auc)
        e_counter+=1
    except FileNotFoundError:
        print("Pre-trained weights not found. Training from scratch.")
        e_counter=0
        best_valid_loss = float('inf')
        best_auc = 0.0
        
    lrs = []
    t_loss = []
    t_auc = []
    v_loss=[]
    v_auc=[]
    epoches=[]
    
    for epoch in range(e_counter,opt.max_epochs):
        epoch_start_time = time.time()
        print()
        print('==================================================================')
        print('-------------Epoch: {}/{}------------'.format(epoch,opt.max_epochs))
        phase = 'train'
        running_loss=0.0
        train_auc=0.0
    
        model.train()
        for idx,batch in enumerate(dataloaders[phase],1):
            images,labels=batch
            images,labels=images.to(device),labels.to(device)
            optimizer.zero_grad()
            
            enc_out,mu,logvar = encoder(images)
            dec_out = decoder(enc_out)    
            
            outputs = model(dec_out)
            loss = l*criterion(outputs, labels)+l*l1_loss (labels,outputs)


#             bs, n_crops, c, h, w = images.size()
#             inputs = images.view(-1, c, h, w)
#             inputs = torch.autograd.Variable(inputs.view(-1, c, h, w))
#             target = torch.autograd.Variable(labels)
#             outputs = model(inputs).view(bs, n_crops, -1).mean(dim=1)
#             loss = l*criterion(outputs, target)+l*l1_loss (target,outputs)

            
            loss.backward()
            optimizer.step()
            running_loss+=loss.item()

            
            preds = outputs.data > 0.5                
            preds = preds.to(torch.float32)
            AUROCs = compute_AUCs(labels, preds)
            AUROC_avg = np.array(AUROCs).mean()
            train_auc += AUROC_avg
            
            
            if (idx)%200 == 0:  
                print(f'{phase}_batch {idx+1}/{len(dataloaders[phase])} Loss: {(running_loss/(idx)):.5f}'
                          f' AUC: {(train_auc /(idx)):.3f}'
                     )

        epoch_loss = running_loss / len(dataloaders[phase])                       
        epoch_auc = train_auc/ len(dataloaders[phase])
        t_auc.append(epoch_auc)
        t_loss.append(epoch_loss)
        print(f'{phase} Epoch Loss: {epoch_loss:.5f} AUC: {epoch_auc: .3f}')
    
        with open(f'{opt.save_path}/train_logs.txt', 'a') as file:
            file.write('epoch: ' + str(epoch) + ',loss: '+ str(epoch_loss) + ',auc: ' + str(epoch_auc) +'\n')

        print('.............')
        phase = 'val'

        running_loss=0.0
        val_auc=0.0

        model.eval()

        with torch.no_grad():
            for idx,batch in enumerate(dataloaders[phase],1):
                images,labels=batch
                images,labels=images.to(device),labels.to(device)
                
                 
                enc_out,mu,logvar = encoder(images)
                dec_out = decoder(enc_out)    

                outputs = model(dec_out)

                loss = l*criterion(outputs, labels)+l*l1_loss (labels,outputs)
#                 bs, n_crops, c, h, w = images.size()
#                 inputs = images.view(-1, c, h, w)
#                 inputs = torch.autograd.Variable(inputs.view(-1, c, h, w))
#                 target = torch.autograd.Variable(labels)
#                 outputs = model(inputs).view(bs, n_crops, -1).mean(dim=1)
#                 loss = l*criterion(outputs, target)+l*l1_loss (target,outputs)


                preds = outputs.data > 0.5               
                preds = preds.to(torch.float32)

                running_loss += loss.item() 
                AUROCs = compute_AUCs(labels, preds)
                AUROC_avg = np.array(AUROCs).mean()
                val_auc += AUROC_avg

                if (idx)%100 == 0:  
                    print(f'{phase}_batch {idx}/{len(dataloaders[phase])} Loss: {(running_loss/((idx))):.5f}'
                              f' AUC: {(val_auc /((idx))):.3f}'
                         )

        epoch_loss = running_loss / len(dataloaders[phase])                        
        epoch_auc = val_auc/ len(dataloaders[phase])
        v_auc.append(epoch_auc)
        v_loss.append(epoch_loss)
        epoches.append(epoch)
        print(f'{phase} Epoch Loss: {epoch_loss:.5f} AUC: {epoch_auc: .3f}')

        state = {
            'epoch': epoch,
            'valid_acc_max': epoch_auc,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
            }

        if epoch_auc > best_auc:
            print('Val auc increased ({:.3f} --> {:.3f}). Saving model ...'.format(best_auc,epoch_auc))
            torch.save(state, os.path.join(opt.model_path, 'I_model.pth'))
            with open(f'{opt.save_path}/val_logs.txt', 'a') as file:
                file.write('epoch: ' + str(epoch) + ',loss: ' + str(epoch_loss)+',auc: ' + str(epoch_auc) +'\n')
            best_auc = epoch_auc
            epoch_time = int(time.time() - epoch_start_time)
            print(f'-----------------Epoch cost time {epoch_time}s--------------------')
        
        lrs.append(optimizer.param_groups[0]["lr"])
        print('learning_rate :',scheduler.get_lr())
        scheduler.step()
        

    plt.plot(lrs)
    
    filepath=os.path.join(opt.save_path, 'LR.png')
    plt.savefig(filepath)
    
    filepath=os.path.join(opt.save_path, 'losses.png')
    plt.title("Training Curve")
    plt.plot(epoches, t_loss, label="Train")
    plt.plot(epoches, v_loss, label="Validation")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend(loc='best')
    plt.savefig(filepath)
    
    filepath=os.path.join(opt.save_path, 'classifier_accuracy.png')
    plt.title("AUC Curve")
    plt.plot(epoches, t_auc, label="Train")
    plt.plot(epoches, v_auc, label="Validation")
    plt.xlabel("Epochs")
    plt.ylabel("ACC")
    plt.legend(loc='best')
    plt.savefig(filepath)

In [27]:
# train_MainNet()

In [28]:
def compute_TAUCs(gt, pred):
    AUROCs = []
    TPRs = FPRs = THs = []
    gt_np = gt
    pred_np = pred
    for i in range(num_classes):
        try:
            AUROCs.append(roc_auc_score(gt_np[:, i], pred_np[:, i]))
        except ValueError:
            pass
    return AUROCs

In [78]:
def test(phase):
    torch.cuda.empty_cache()
    opt = Config()
    model.to(device)
    P=[]
    L=[]
    model.eval()
    for batch in tqdm(dataloaders[phase]):
        images,labels=batch
        images,labels=images.to(device),labels.to(device)
        outputs = model(images)
        preds = outputs.data > 0.5               
        preds = preds.to(torch.float32)

        preds=preds.cpu().numpy().flatten()
        labels=labels.cpu().numpy().flatten()

        P.append(preds)
        L.append(labels)
        
    P = np.array(P)
    L = np.array(L)
    AUROCs = compute_TAUCs(L,P)
    AUROC_avg = np.array(AUROCs).mean()
    print(f'AUC on {phase} set: ',AUROC_avg)

In [79]:
phase = ['val','covid']
# phase=['covid']
for p in phase:
    test(p)


  0%|          | 0/392 [00:00<?, ?it/s]

AUC on val set:  0.5043762591195272


100%|██████████| 392/392 [00:17<00:00, 22.63it/s]

AUC on covid set:  0.5255940295223491


In [66]:
# gt_np =np.array([[1,1,0,1], [1, 1, 0,0], [0, 0 ,0,1]])
# pred_np =np.array([[1,1,0,1], [1, 1, 0,0], [0, 0 ,0,1]])
# AUROCs = []
# for i in range(4):
#     try:
#         AUROCs.append(roc_auc_score(gt_np[:, i], pred_np[:, i]))
#     except:
#         pass
# print(AUROCs)
# AUROC_avg = np.array(AUROCs).mean()
# print(AUROC_avg)